In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 32
refRatio = 2

deriv = 'CD'
order = 2

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max


In [4]:
def CDStencil(orderIn):
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    loops = int(order / 2)
        
    coefs = np.zeros(loops)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    rCell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    lCell = rCell + 0
    lCell[1::2] = -lCell[1::2]
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(loops)]
    for k in range(loops):
        rCellNew = rCell * deltaXFunc(k + 1)
        lCellNew = lCell * deltaXFunc(k + 1)
        tExp[k] = rCellNew - lCellNew
        if(k == 0):
            coefs[k] = 1
        else:
            scale = tExp[k][(2 * k) + 1] / tExp[k - 1][(2 * k) + 1]
            coefs = scale * coefs
            coefs[k] = (-1)**k
        
    stenc[:loops] = coefs[::-1]
    stenc[loops + 1:] = -coefs
    return stenc

In [5]:
def UDStencil(order):
    coefs = np.zeros(order + 1)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    cell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(order + 1)]
    for k in range(order + 1):
        cellNew = cell * deltaXFunc(k)
        tExp[k] = cellNew
    
    tExp = np.asarray(tExp).transpose()
    mat = np.zeros((order, order), float)
    vec = np.zeros(order, float)
    mat[0, :] = tExp[0, :order]
    mat[1:, :] = tExp[2:, :order]
    vec[0] = tExp[0, order]
    vec[1:] = tExp[2:, order]
    vec = -vec
    stenc[order] = 1
    stenc[:order] = LA.inv(mat) @ vec
    val = (tExp @ stenc)[1]
    
#     UNCOMMENT THIS LINE!!!!!
#     stenc = stenc / val



#     print(tExp)
#     print('')
#     print(mat)
#     print('')
#     print(vec)
#     print('')
#     print(val)
#     print('')
#     stenc[:order] = coefs[::-1]
#     stenc[order + 1:] = -coefs
    return stenc

In [6]:
def DDStencil(order):
    stenc = -UDStencil(order)[::-1]
    return stenc

In [7]:
def CentStenc(order, diff):
    if (diff == 'C' or diff == 'CD'):
        stenc = CDStencil(order)
    else:
        piece = np.zeros(order, float)
        if (diff == 'U' or diff == 'UD'):
            stenc = UDStencil(order)
            stenc = np.append(piece, stenc)
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDStencil(order)
                stenc = np.append(stenc, piece)
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    return stenc

In [8]:
stenc1 = CentStenc(order, 'C')
stenc2 = CentStenc(order, 'U')
stenc3 = CentStenc(order, 'D')
ghostCell = GTT.GhostCellStencil(order, 1.5)
print(stenc1)
print('')
print(stenc2)
print('')
print(stenc3)
print('')
print(ghostCell)
print('')



START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1. ]
END BoundVals() FUNC!

polyInterp: [ 0.5        -1.16666667  1.66666667]

[ 1.  0. -1.]

[ 0.  0.  3. -4.  1.]

[-1.  4. -3.  0.  0.]

[ 0.5        -1.16666667  1.66666667]



In [9]:
def CentGhost(order, x_0, diff):
    ghostCell = GTT.GhostCellStencil(order, x_0)
    bounds = GTT.BoundVals(order, x_0)
    lenL = np.shape(bounds[bounds < 0])[0]
    lenR = np.shape(bounds[bounds > 0])[0]
    print(lenL, lenR)
    if (x_0 > 0):
        if (lenR == lenL + 1):
            ghostCent = ghostCell
        else:
            addOn = abs(lenL - lenR + 1)
            piece = np.zeros(addOn, float)
            if (lenL < lenR + 1):
                ghostCent = np.append(piece, ghostCell)
            else:
                ghostCent = np.append(ghostCell, piece)
    else:
        if (x_0 < 0):
            a = 1
        else:
            errorMess = 'Argument x_0 cannot be 0!'
    return ghostCent

In [10]:
centGhost = CentGhost(order, 0.5, deriv)
print(centGhost)


START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1. ]
END BoundVals() FUNC!

polyInterp: [-0.25  0.75  0.5 ]


START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1. ]
END BoundVals() FUNC!

2 1
[-0.25  0.75  0.5   0.    0.  ]


In [11]:
def SpaceDeriv(omega, order, diff):
    
    # Extract info from omega.
    hs = omega.h
    degFreed = omega.degFreed
    
    # Create empty matrizes to fill later.
    blockMat = np.zeros((degFreed, degFreed), float)
    zeroMat = np.zeros(degFreed, float)
    
    # Interpolate ghost cells.
    ghostStencL = GTT.GhostCellStencil(order, -0.5)
    ghostStencR = GTT.GhostCellStencil(order, 0.5)
    
    # Create common constituents of row pieces.
    rightCell = zeroMat + 0
    leftCell = zeroMat + 0
    ghostL = zeroMat + 0
    ghostR = zeroMat + 0
    ghostL[:order+1] = ghostStencL
    ghostR[:order+1] = ghostStencR
    nrollL = int(np.ceil((order + 1) / 3.))
    nrollR = int(order - nrollL)
    ghostL = np.roll(ghostL, -nrollL)
    ghostR = np.roll(ghostR, -nrollR)
    
    print('HELLO?')
    print(ghostL)
    print(ghostR)
    print('')
    
    # Define distinct row pieces between upwind and center difference.
    if (diff == 'U'):
        rightCell[0] = 1
        leftCell[-1] = 1
        
        cf1v2 = rightCell + 0
        cf2v1 = ghostL
        
        fc1v2 = rightCell + 0
        fc2v1 = zeroMat + 0
        fc2v1[-1] = 0.5
        fc2v1[-2] = 0.5
        fc2v2 = rightCell + 0
        
#         hMat = StepMatrix(omega)
    else:
        if (diff == 'D'):
            rightCell[1] = 1
            leftCell[0] = 1
            
            cf1v2 = zeroMat + 0
            cf1v2[1] = 0.5
            cf1v2[2] = 0.5
            cf2v1 = leftCell + 0
            
            fc1v2 = ghostR
            fc2v1 = leftCell + 0
            
#             hMat = StepMatrix(omega)
        else:
            rightCell[1] = 1
            leftCell[-1] = 1

            cf1v2 = zeroMat + 0
            cf1v2[1] = 0.5
            cf1v2[2] = 0.5
            cf2v1 = ghostL
            
            fc1v2 = ghostR
            fc2v1 = zeroMat + 0
            fc2v1[-1] = 0.5
            fc2v1[-2] = 0.5

#             hMat = 0.5 * StepMatrix(omega)
    
    # Define common row pieces between upwind and center difference.
    cf1v1 = leftCell + 0 # New add.
    cf2v2 = rightCell + 0
    
    fc1v1 = leftCell + 0 # New add.
    fc2v2 = rightCell + 0
    
    # Define rows.
    default = rightCell - leftCell
    cf1 = cf1v2 - cf1v1
    cf2 = cf2v2 - cf2v1
    fc1 = fc1v2 - fc1v1
    fc2 = fc2v2 - fc2v1
    
    # Create vector containing intergrid boundary locations.
    spots = np.roll(hs, -1) - hs
    print('spots is')
    print(spots)
    
    # Fill matrix.
    for k in range(degFreed):
        print('ITERATION ' + str(k) +':')
        if (np.roll(spots, 1 - k)[0] > 0):
            print('CONDITION 1:')
            print(fc2)
            print('')
            row = fc2
        else:
            if (np.roll(spots, -k)[0] < 0): # This one's okay.
                print('CONDITION 2:')
                print(cf1)
                print('')
                row = cf1
                
            else:
                if (np.roll(spots, 1 - k)[0] < 0): # This one's okay.
                    print('CONDITION 3:')
                    print(cf2)
                    print('')
                    row = cf2 #fc1
                    
                else:
                    if (spots[k] > 0): # This one's okay.
                        print('CONDITION 4:')
                        print(fc1)
                        print('')
                        row = fc1 #cf2
                    else:
                        print('CONDITION 5:')
                        print(default)
                        print('')
                        row = default
        blockMat[k, :] = np.roll(row, k)

#     blockMat = hMat @ blockMat

    return blockMat


In [12]:
derivOp = SpaceDeriv(omega, order, 'CD')


START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.   0.   0.5  1. ]
END BoundVals() FUNC!

polyInterp: [ 0.5   0.75 -0.25]


START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1. ]
END BoundVals() FUNC!

polyInterp: [-0.25  0.75  0.5 ]

HELLO?
[ 0.75 -0.25  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.5 ]
[ 0.75  0.5   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -0.25]

spot

In [13]:
print(derivOp)

[[-0.75  1.25  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   -0.5 ]
 [-1.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.   -1.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  

In [14]:
A = np.arange(64).reshape(8, 8)
print(A)

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]


In [44]:
order = 10
for o in range(order):
    print('START: ' + str(int(o + 1)) + ':')
    ghostCell = GTT.GhostCellStencil(int(o + 1), 0.5)
    print('END: ' + str(int(o + 1)) + ':')
    print('')

START: 1:

START BoundVals() FUNC!
n_c: 1
n_f: 1
bounds: [-1.  -0.5  0.   0.5]
rm: [1]
bounds: [-1.   0.   0.5]
bounds: [-0.5 -0.   1. ]
END BoundVals() FUNC!

polyInterp: [0.33333333 0.66666667]

END: 1:

START: 2:

START BoundVals() FUNC!
HERE!
n_c: 1
n_f: 2
bounds: [-1.  -0.5  0.   0.5  1. ]
rm: [1]
bounds: [-1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1. ]
END BoundVals() FUNC!

polyInterp: [-0.25  0.75  0.5 ]

END: 2:

START: 3:

START BoundVals() FUNC!
n_c: 2
n_f: 2
bounds: [-2.  -1.5 -1.  -0.5  0.   0.5  1. ]
rm: [1, 3]
bounds: [-2.  -1.   0.   0.5  1. ]
bounds: [-1.  -0.5 -0.   1.   2. ]
END BoundVals() FUNC!

polyInterp: [-0.125  0.475  0.7   -0.05 ]

END: 3:

START: 4:

START BoundVals() FUNC!
n_c: 2
n_f: 3
bounds: [-2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5]
rm: [1, 3]
bounds: [-2.  -1.   0.   0.5  1.   1.5]
bounds: [-1.5 -1.  -0.5 -0.   1.   2. ]
END BoundVals() FUNC!

polyInterp: [ 0.08571429 -0.41428571  0.78571429  0.57142857 -0.02857143]

END: 4:

START: 5:

START BoundVa

In [109]:
x_0 = 2

for i in range(order):
    o = int(i + 1)
    print('START: ' + str(o) + ':')
    
    
    a = abs(int(x_0 / 0.5))
    cells = o + 1
    if (cells <= a):
        n_c = cells
    else:
        n_c = a
        cells_new = cells - a
        if ((cells_new + 1) % 3 == 0):
            print('HERE!')
            n_c_add = int(np.floor((cells_new + 1) / 3) - 1)
        else:
            n_c_add = int(np.floor((cells_new + 1) / 3))
        n_c = n_c + n_c_add
    
    n_f = cells - n_c
    print('n_c =', n_c)
    print('n_f =', n_f)
    if (n_f != 0):
        bounds = np.linspace(-n_c, n_f / 2., num = (2 * n_c) + n_f + 1)
#         print(bounds)
        rm = [(2 * k) + 1 for k in range(n_c)]
        bounds = np.delete(bounds, rm)
        if (x_0 > 0):
            bounds = -bounds[::-1]
    else:
        if (x_0 % 1 == 0):
            bounds = np.arange(n_c + 1) + (int(x_0) - int((n_c + 1) / 2))
        else:
            bounds = np.arange(n_c + 1) + (int(x_0) - int(n_c / 2))
            
    
    print('bounds =', bounds)
    print('END: ' + str(o) + ':')
    print('')

START: 1:
n_c = 2
n_f = 0
bounds = [1 2 3]
END: 1:

START: 2:
n_c = 3
n_f = 0
bounds = [0 1 2 3]
END: 2:

START: 3:
n_c = 4
n_f = 0
bounds = [0 1 2 3 4]
END: 3:

START: 4:
n_c = 4
n_f = 1
bounds = [-0.5 -0.   1.   2.   3.   4. ]
END: 4:

START: 5:
HERE!
n_c = 4
n_f = 2
bounds = [-1.  -0.5 -0.   1.   2.   3.   4. ]
END: 5:

START: 6:
n_c = 5
n_f = 2
bounds = [-1.  -0.5 -0.   1.   2.   3.   4.   5. ]
END: 6:

START: 7:
n_c = 5
n_f = 3
bounds = [-1.5 -1.  -0.5 -0.   1.   2.   3.   4.   5. ]
END: 7:

START: 8:
HERE!
n_c = 5
n_f = 4
bounds = [-2.  -1.5 -1.  -0.5 -0.   1.   2.   3.   4.   5. ]
END: 8:

START: 9:
n_c = 6
n_f = 4
bounds = [-2.  -1.5 -1.  -0.5 -0.   1.   2.   3.   4.   5.   6. ]
END: 9:

START: 10:
n_c = 6
n_f = 5
bounds = [-2.5 -2.  -1.5 -1.  -0.5 -0.   1.   2.   3.   4.   5.   6. ]
END: 10:



In [67]:
print(int(3.5))

3
